In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
import numpy as np
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
import pandas as pd
import random
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info import Statevector
//test

In [2]:
def circuit_run(x1,x2,x3):
    qc = QuantumCircuit(1)
    qc.u(x1*np.pi/2,-np.pi/2,np.pi/2,0)
    qc.u(x2*np.pi/2,-np.pi/2,np.pi/2,0)
    qc.u(x3*np.pi/2,-np.pi/2,np.pi/2,0)
    
    output = Statevector.from_instruction(qc).to_dict()
    p0,p1 = 0,0
    if '0' in output.keys():
        p0 = np.round(np.abs(output['0'])**2,decimals=4)
    if '1' in output.keys():
        p1 = np.round(np.abs(output['1'])**2,decimals=4)
    
    if p0==1:
        return 0
    elif p1==1:
        return 1
    else:
        return 'x'

In [3]:
def hamming_distance(class_dist,quant_dist):
    count = 0
    for x,y in zip(class_dist,quant_dist):
        if y!='x':
            count += (x!=y)
    return count

In [ ]:
#quantum
quant_dist = []
for x1 in range(4):
    for x2 in range(4):
        for x3 in range(4):
            quant_dist.append(circuit_run(x1,x2,x3))
quant_dist

<h2>Appendix B proof</h2>

In [45]:
def weighted_sum(p0,p1,p2,p3,p4,p5,p6,p7):
    sum = 0
    for u in range(2):
        for v in range(2):
            for w in range(2):
                if (u+v+w)%2 == 0:
                    for b1 in range(2):
                        for b2 in range(2):
                            for b3 in range(2):
                                    sum += (-1)**(p1*b1 + p2*b2 + p3*b3) * (1j)**(u+v+w) * (-1)**(p0 + u*p4 + v*p5 + w*p6+ p7*u*v)
    return np.real(sum)

best = 0
best_weights = (0,0,0,0,0,0,0,0,0)
for p0 in range(2):
    for p1 in range(2):
        for p2 in range(2):
            for p3 in range(2):
                for p4 in range(2):
                    for p5 in range(2):
                        for p6 in range(2):
                            for p7 in range(2):
                                    if weighted_sum(p0,p1,p2,p3,p4,p5,p6,p7) >= best:
                                        best = weighted_sum(p0,p1,p2,p3,p4,p5,p6,p7)
                                        best_weights = (p0,p1,p2,p3,p4,p5,p6,p7)
                                        # print((p0,p1,p2,p3,p4))
best_weights
# best

(0, 0, 0, 0, 1, 1, 0, 1)

<h2>Measurement correlations with two bits input</h2>

In [6]:
def build_circuit(u1,u2,v1,v2,w1,w2):
    qreg_q = QuantumRegister(6, 'q')
    circuit = QuantumCircuit(qreg_q)
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.h(qreg_q[2])
    circuit.h(qreg_q[3])
    circuit.h(qreg_q[4])
    circuit.h(qreg_q[5])
    circuit.cz(qreg_q[5], qreg_q[0])
    circuit.cz(qreg_q[1], qreg_q[0])
    circuit.cz(qreg_q[2], qreg_q[1])
    circuit.h(qreg_q[0])
    circuit.cz(qreg_q[3], qreg_q[2])
    if u2 == 1:
        circuit.s(qreg_q[1])
        circuit.z(qreg_q[1])
    circuit.h(qreg_q[1])
    circuit.cz(qreg_q[4], qreg_q[3])
    circuit.h(qreg_q[2])
    circuit.cz(qreg_q[5], qreg_q[4])
    if v2 == 1:
        circuit.s(qreg_q[3])
        circuit.z(qreg_q[3])
    circuit.h(qreg_q[3])
    circuit.h(qreg_q[4])
    if w2 == 1:
        circuit.s(qreg_q[5])
        circuit.z(qreg_q[5])
    circuit.h(qreg_q[5])

    if u1 == 1:
        circuit.x(qreg_q[1])
    if v1 == 1:
        circuit.x(qreg_q[3])
    if w1 == 1:
        circuit.x(qreg_q[5])

    circuit.measure_all()
    return circuit

circuit = build_circuit(1,0,0,1,1,1)
circuit.draw()

┌───┐      ┌───┐                                    ░ ┌─┐            »
   q_0: ┤ H ├─■──■─┤ H ├────────────────────────────────────░─┤M├────────────»
        ├───┤ │  │ └───┘┌───┐┌───┐                          ░ └╥┘┌─┐         »
   q_1: ┤ H ├─┼──■───■──┤ H ├┤ X ├──────────────────────────░──╫─┤M├─────────»
        ├───┤ │      │  └───┘├───┤                          ░  ║ └╥┘┌─┐      »
   q_2: ┤ H ├─┼──────■────■──┤ H ├──────────────────────────░──╫──╫─┤M├──────»
        ├───┤ │           │  └───┘┌───┐┌───┐┌───┐           ░  ║  ║ └╥┘┌─┐   »
   q_3: ┤ H ├─┼───────────■────■──┤ S ├┤ Z ├┤ H ├───────────░──╫──╫──╫─┤M├───»
        ├───┤ │                │  └───┘├───┤└───┘           ░  ║  ║  ║ └╥┘┌─┐»
   q_4: ┤ H ├─┼────────────────■────■──┤ H ├────────────────░──╫──╫──╫──╫─┤M├»
        ├───┤ │                     │  ├───┤┌───┐┌───┐┌───┐ ░  ║  ║  ║  ║ └╥┘»
   q_5: ┤ H ├─■─────────────────────■──┤ S ├┤ Z ├┤ H ├┤ X ├─░──╫──╫──╫──╫──╫─»
        └───┘                          └───┘└───┘└───┘└───┘ ░  ║  ║  ║  ║  ║ »
meas: 6/═══════════════════════════════════════════════════════╩══╩══╩══╩══╩═»
                                                               0  1  2  3  4 »
«           
«   q_0: ───
«           
«   q_1: ───
«           
«   q_2: ───
«           
«   q_3: ───
«           
«   q_4: ───
«        ┌─┐
«   q_5: ┤M├
«        └╥┘
«meas: 6/═╩═
«         5

In [ ]:
def compute_corel(u1,u2,v1,v2,w1,w2):
    circuit = build_circuit(u1,u2,v1,v2,w1,w2)
    backend = Aer.get_backend('qasm_simulator')
    backend.set_option("shots",1)
    res = backend.run(circuit).result().get_counts().popitem()[0]
    
    mu= (-1)**int(res[5-1])
    mv= (-1)**int(res[5-3])
    mw= (-1)**int(res[5-5])
    mB = (-1)**int(res[5-0])
    mL = (-1)**int(res[5-2])
    mR = (-1)**int(res[5-4])
    # print(mB*mL*mR)

    return np.real( (-1)**(u1+v1+w1) * (1j)**(u2+v2+w2) * mu*mv*mw*(mR**u2)*(mL**w2)*(mB**v2) )

all = []
for u1 in range(2):
    for v1 in range(2):
        for w1 in range(2):
            for u2 in range(2):
                for v2 in range(2):
                    for w2 in range(2):
                        if (u2+v2+w2)%2==0:
                            r= compute_corel(u1,u2,v1,v2,w1,w2)
                            print(r)
                            all.append(r)
np.sum(all)

<h2>Classical circuit correlations optimization</h2>

<h3>1 : Three bits functions + one bit functions</h3>
-Weird modelisation

-Too long -> cannot conclude

In [ ]:
#Full classical circuit sim

def three_bits_function(b,x,w):
    return (w[0]+w[1]*x[0]+w[2]*x[1]+w[3]*b)%2

def one_bit_function(b,p):
    return (p[0] + p[1]*b)%2

def classical_circuit_corel(x1,x2,x3,w,p):

    B = three_bits_function(0,x1,w[0])
    b1 = one_bit_function(B,p[0])

    u = three_bits_function(b1,x1,w[1])
    b2 = one_bit_function(u,p[1])

    L = three_bits_function(b2,x2,w[2])
    b3 = one_bit_function(L,p[2])

    v = three_bits_function(b3,x2,w[3])
    b4 = one_bit_function(v,p[3])

    R = three_bits_function(b4,x3,w[4])
    b5 = one_bit_function(R,p[4])

    w_res = three_bits_function(b5,x3,w[5])
    y = one_bit_function(w_res,p[5])

    mu = (-1)**u
    mv = (-1)**v
    mw = (-1)**w_res
    mB = (-1)**B
    mL = (-1)**L
    mR = (-1)**R

    if mB*mL*mR != 1:
        return False
    
    return (mu,mv,mw,mB,mL,mR)

def distribution_classical_circuit_corel(w,p):
    x_val = [(0,0),(0,1),(1,0),(1,1)]

    results = []
    for x1 in x_val:
        for x2 in x_val:
            for x3 in x_val:
                r = classical_circuit_corel(x1,x2,x3,w,p)
                if not r: #The parameter are not good because they do not respect correlation (1) : mB*mL*mR=1
                    return False
                
                if (x1[1]+x2[1]+x3[1])%2==0: #Now we can check correlation (2)
                    mu,mv,mw,mB,mL,mR = r
                    cor = int(np.real((1j)**(x1[1]+x2[1]+x3[1]) * mu*mv*mw*(mR**x1[1])*(mL**x2[1])*(mB**x3[1]) * (-1)**(x1[0]+x2[0]+x3[0]) ))
                    if cor==1:
                        results.append(r)
    return len(results)

def classical_circuit_output(x1,x2,x3,w,p):

    B = three_bits_function(0,x1,w[0])
    b1 = one_bit_function(B,p[0])

    u = three_bits_function(b1,x1,w[1])
    b2 = one_bit_function(u,p[1])

    L = three_bits_function(b2,x2,w[2])
    b3 = one_bit_function(L,p[2])

    v = three_bits_function(b3,x2,w[3])
    b4 = one_bit_function(v,p[3])

    R = three_bits_function(b4,x3,w[4])
    b5 = one_bit_function(R,p[4])

    w_res = three_bits_function(b5,x3,w[5])
    y = one_bit_function(w_res,p[5])

    return y

def distribution_classical_circuit_output(w,p):
    x_val = [(0,0),(0,1),(1,0),(1,1)]

    results = []
    for x1 in x_val:
        for x2 in x_val:
            for x3 in x_val:
                results.append(classical_circuit_output(x1,x2,x3,w,p))
    return results

possibilities_w = [[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1],[0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]]
possibilities_p = [[0, 0], [0, 1], [1, 0], [1, 1]]

best = 0
best_ham = 100

for w0 in possibilities_w:
    for w1 in possibilities_w:
        for w2 in possibilities_w:
            for w3 in possibilities_w:
                for w4 in possibilities_w:
                    for w5 in possibilities_w:

                        w = [w0,w1,w2,w3,w4,w5]
                        p = [(0,1), (0,1), (0,1), (0,1), (0,1), (0,1)]
                        val = distribution_classical_circuit_corel(w,p)
                        if val and val > best:
                            best = val
                            print(val)
    
                            # class_dist = distribution_classical_circuit_output(w,p)
                            # ham = hamming_distance(class_dist,quant_dist)
                            # if ham<best_ham:
                            #     best_ham = ham
                            #     print("ham distance {}".format(ham))

16
24


<h3>2 : All differents gates that output two bits each</h3>

-Most powerful modelisation

-<b>NO ADVANTAGE</b>

-Too long

In [63]:
#All different gates but x does not enter in B,L,R gates and no one bit function
#Full classical circuit sim

def three_bits_function_new(b,x,w1,w2):
    return ( (w1[0]+w1[1]*x[0]+w1[2]*x[1]+w1[3]*b)%2, (w2[0]+w2[1]*x[0]+w2[2]*x[1]+w2[3]*b)%2)

def one_bit_function_new(b,p1,p2):
    return ( (p1[0] + p1[1]*b)%2, (p2[0] + p2[1]*b)%2)

def classical_circuit_corel_new2(x1,x2,x3,w,p):

    B,b1 = one_bit_function_new(0,p[0],p[1])
    u,b2 = three_bits_function_new(b1,x1,w[0],w[1])
    L,b3 = one_bit_function_new(b2,p[2],p[3])
    v,b4 = three_bits_function_new(b3,x2,w[2],w[3])
    R,b5 = one_bit_function_new(b4,p[4],p[5])
    w_res,y = three_bits_function_new(b5,x3,w[4],w[5])

    mu = (-1)**u
    mv = (-1)**v
    mw = (-1)**w_res
    mB = (-1)**B
    mL = (-1)**L
    mR = (-1)**R

    if mB*mL*mR != 1:
        return False
    
    return (mu,mv,mw,mB,mL,mR)

def distribution_classical_circuit_corel_new2(w,p):
    x_val = [(0,0),(0,1),(1,0),(1,1)]

    results = []
    for x1 in x_val:
        for x2 in x_val:
            for x3 in x_val:
                r = classical_circuit_corel_new2(x1,x2,x3,w,p)
                if not r: #The parameter are not good because they do not respect correlation (1) : mB*mL*mR=1
                    return False
                
                if (x1[1]+x2[1]+x3[1])%2==0: #Now we can check correlation (2)
                    mu,mv,mw,mB,mL,mR = r
                    cor = int(np.real((1j)**(x1[1]+x2[1]+x3[1]) * mu*mv*mw*(mR**x1[1])*(mL**x2[1])*(mB**x3[1]) * (-1)**(x1[0]+x2[0]+x3[0]) ))
                    if cor==1:
                        results.append(r)
    return len(results)

possibilities_w = [[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1],[0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]]
possibilities_p = [[0, 0], [0, 1], [1, 0], [1, 1]]

# best = 0
# best_ham = 100

# for w0 in possibilities_w:
#     for w1 in possibilities_w:
#         for w2 in possibilities_w:
#             for w3 in possibilities_w:
#                 for w4 in possibilities_w:
#                     for w5 in possibilities_w:
                            
#                         p0 = random.choice(possibilities_p)
#                         p1 = random.choice(possibilities_p)
#                         p2 = random.choice(possibilities_p)
#                         p3 = random.choice(possibilities_p)
#                         p4 = random.choice(possibilities_p)
#                         p5 = random.choice(possibilities_p)

#                         w = [w0,w1,w2,w3,w4,w5]
#                         p = [p0,p1,p2,p3,p4,p5]
#                         val = distribution_classical_circuit_corel_new2(w,p)
#                         if val:
#                             class_dist = distribution_classical_circuit_output_new2(w,p)
#                             ham = hamming_distance(class_dist, quant_dist)
#                             if ham < best_ham:
#                                 print(ham)
#                                 best_ham = ham

best_w = [[0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 0, 1]]
best_p = [[1, 0], [1, 0], [0, 1], [1, 1], [0, 1], [1, 1]]

distribution_classical_circuit_corel_new2(best_w,best_p)

#Really long and NOT WORKING

32

<h3>3 : Only two types of gates</h3>

Correspond $H$ and $J(\alpha)$ gates

-Good modelisation but maybe not powerfull enough

-$H$ gates may depend or not on $x_i$

-Quantum advantage is observed anyway

In [57]:
#Assumption the gate are similar (two types) and no one bit function
#Full classical circuit sim

def classical_circuit_corel_new(x1,x2,x3,w):

    B,b1 = three_bits_function_new(0,x1,w[0],w[1])
    u,b2 = three_bits_function_new(b1,x1,w[2],w[3])
    L,b3 = three_bits_function_new(b2,x2,w[0],w[1])
    v,b4 = three_bits_function_new(b3,x2,w[2],w[3])
    R,b5 = three_bits_function_new(b4,x3,w[0],w[1])
    w_res,y = three_bits_function_new(b5,x3,w[2],w[3])

    mu = (-1)**u
    mv = (-1)**v
    mw = (-1)**w_res
    mB = (-1)**B
    mL = (-1)**L
    mR = (-1)**R

    if mB*mL*mR != 1:
        return False
    
    return (mu,mv,mw,mB,mL,mR)

def distribution_classical_circuit_corel_new(w):
    x_val = [(0,0),(0,1),(1,0),(1,1)]

    results = []
    for x1 in x_val:
        for x2 in x_val:
            for x3 in x_val:
                r = classical_circuit_corel_new(x1,x2,x3,w)
                if not r: #The parameter are not good because they do not respect correlation (1) : mB*mL*mR=1
                    return False
                
                if (x1[1]+x2[1]+x3[1])%2==0: #Now we can check correlation (2)
                    mu,mv,mw,mB,mL,mR = r
                    cor = int(np.real((1j)**(x1[1]+x2[1]+x3[1]) * mu*mv*mw*(mR**x1[1])*(mL**x2[1])*(mB**x3[1]) * (-1)**(x1[0]+x2[0]+x3[0]) ))
                    if cor==1:
                        results.append(r)
    return len(results)

def classical_circuit_output_new(x1,x2,x3,w):

    B,b1 = three_bits_function_new(0,x1,w[0],w[1])
    u,b2 = three_bits_function_new(b1,x1,w[2],w[3])
    L,b3 = three_bits_function_new(b2,x2,w[0],w[1])
    v,b4 = three_bits_function_new(b3,x2,w[2],w[3])
    R,b5 = three_bits_function_new(b4,x3,w[0],w[1])
    w_res,y = three_bits_function_new(b5,x3,w[2],w[3])

    return y

def distribution_classical_circuit_output_new(w):
    x_val = [(0,0),(0,1),(1,0),(1,1)]

    results = []
    for x1 in x_val:
        for x2 in x_val:
            for x3 in x_val:
                results.append(classical_circuit_output_new(x1,x2,x3,w))
    return results

possibilities_w = [[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1],[0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]]

best = 0
best_ham = 100

cpt = 0
for w0 in possibilities_w:
    for w1 in possibilities_w:
        # if w0[1]==0 and w0[2]==0 and w1[1]==0 and w1[2]==0:
            for w2 in possibilities_w:
                for w3 in possibilities_w:
                    cpt +=1

                    w = [w0,w1,w2,w3]
                    val = distribution_classical_circuit_corel_new(w)
                    if val and val > best:
                        best = val
                        print(val)
                    
                        # class_dist = distribution_classical_circuit_output_new(w)
                        # ham = hamming_distance(class_dist,quant_dist)
                        # if ham<best_ham:
                        #     best_ham = ham
                        #     print("ham distance {}".format(ham))

16
24


<h3>4 : One type of gate, each output 3 bits</h3>

-Most similar to the original scheme

-Least powerful

-Advantage for quantum

In [11]:
#3 sames gates (one type) outputing 3 bits each --> more limited than 2 types
def three_bits_function_new3(b,x,w1,w2,w3):
    return ( (w1[0]+w1[1]*x[0]+w1[2]*x[1]+w1[3]*b)%2, (w2[0]+w2[1]*x[0]+w2[2]*x[1]+w2[3]*b)%2, (w3[0]+w3[1]*x[0]+w3[2]*x[1]+w3[3]*b)%2)

def classical_circuit_corel_new3(x1,x2,x3,w):

    B,u,b1 = three_bits_function_new3(0,x1,w[0],w[1],w[2])
    L,v,b2 = three_bits_function_new3(b1,x2,w[0],w[1],w[2])
    R,w_res,y = three_bits_function_new3(b2,x3,w[0],w[1],w[2])

    mu = (-1)**u
    mv = (-1)**v
    mw = (-1)**w_res
    mB = (-1)**B
    mL = (-1)**L
    mR = (-1)**R

    if mB*mL*mR != 1:
        return False
    
    return (mu,mv,mw,mB,mL,mR)

def distribution_classical_circuit_corel_new3(w):
    x_val = [(0,0),(0,1),(1,0),(1,1)]

    results = []
    for x1 in x_val:
        for x2 in x_val:
            for x3 in x_val:
                r = classical_circuit_corel_new3(x1,x2,x3,w)
                if not r: #The parameter are not good because they do not respect correlation (1) : mB*mL*mR=1
                    return False
                
                if (x1[1]+x2[1]+x3[1])%2==0: #Now we can check correlation (2)
                    mu,mv,mw,mB,mL,mR = r
                    cor = int(np.real((1j)**(x1[1]+x2[1]+x3[1]) * mu*mv*mw*(mR**x1[1])*(mL**x2[1])*(mB**x3[1]) * (-1)**(x1[0]+x2[0]+x3[0]) ))
                    if cor==1:
                        results.append(r)
    return len(results)

possibilities_w = [[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1],[0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]]
possibilities_p = [[0, 0], [0, 1], [1, 0], [1, 1]]

best = 0

for w0 in possibilities_w:
    for w1 in possibilities_w:
        for w2 in possibilities_w:
            w = [w0,w1,w2,w3]
            val = distribution_classical_circuit_corel_new3(w)
            if val and val > best:
                best = val
                print(val)

16
24


<h2>Test of output propagation correction</h2>

In [134]:
def three_bits_combination(u,v,w_res,w):
    return (w[0]*u+w[1]*v+w[2]*w_res)%2

def four_bits_function(b,x,w):
    return (w[0]+w[1]*x[0]+w[2]*x[1]+w[3]*x[2]+w[4]*b)%2

def classical_circuit_output_new2(x1,x2,x3,w,p):

    B,b1 = one_bit_function_new(0,p[0],p[1])
    u,b2 = three_bits_function_new(b1,x1,w[0],w[1])
    L,b3 = one_bit_function_new(b2,p[2],p[3])
    v,b4 = three_bits_function_new(b3,x2,w[2],w[3])
    R,b5 = one_bit_function_new(b4,p[4],p[5])
    w_res,y = three_bits_function_new(b5,x3,w[4],w[5])

    mu = (-1)**u
    mv = (-1)**v
    mw = (-1)**w_res
    mB = (-1)**B
    mL = (-1)**L
    mR = (-1)**R

    return (y,mu,mv,mw,mB,mL,mR)

def distribution_classical_circuit_output_new2(w,p,quant_dist,w2):
    x_val = [(0,0),(0,1),(1,0),(1,1)]

    results = []
    for i1,x1 in enumerate(x_val):
        for i2,x2 in enumerate(x_val):
            for i3,x3 in enumerate(x_val):
                y,mu,mv,mw,mB,mL,mR = classical_circuit_output_new2(x1,x2,x3,w,p)
                if quant_dist[i1*16+i2*4+i3]!='x' and quant_dist[i1*16+i2*4+i3] != y:
                    pass
                results.append(y)
    return results

In [140]:
bad_w = [[0, 1, 1, 1], [0, 1, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 0, 1]]
bad_p = [[1, 0], [0, 0], [0, 1], [1, 1], [0, 1], [1, 1]]

# p = best_p
# w = best_w

p = bad_p
w = bad_w

distribution_classical_circuit_corel_new2(w,p)
   
# class_dist = distribution_classical_circuit_output_new2(w,p,quant_dist,w2)
# ham = hamming_distance(class_dist, quant_dist)
# ham

16